In [ ]:
# MeCabとIPAdicのインストール
!apt-get -y install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 ipadic

In [7]:
import ipadic
import MeCab
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from typing import Union, List
import os

# pandasの小数点以下の桁数を3に設定
pd.options.display.precision = 3

# MeCabで分かち書き
def wakati(text: str) -> str:
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")
    return tagger.parse(text).strip()

# --- BLEU計算関数 ---
def compute_bleu(predictions: List[str], references: List[str]) -> dict:
    """
    corpus_bleu 用に形式を整える
    predictions: 生成文リスト
    references: 参照文リスト（単一参照なら1文のリスト）
    """
    # 分かち書き
    pred_wakati = [p.split() for p in predictions]
    ref_wakati = [[r.split()] for r in references]  # corpus_bleuは参照文のリストを二重リストにする

    smoothing = SmoothingFunction().method1
    score = corpus_bleu(ref_wakati, pred_wakati, smoothing_function=smoothing)

    return {"BLEU": round(score, 3)}

# --- 入力処理関数 ---
def load_data(input_data: Union[str, List[str]], tagger: MeCab.Tagger) -> List[str]:
    data_list = []
    if isinstance(input_data, str):
        if os.path.exists(input_data):
            print(f"✅ ファイル '{input_data}' からデータを読み込みます。")
            with open(input_data, 'r', encoding='utf-8') as f:
                data_list = [line.strip() for line in f if line.strip()]
        else:
            print(f"💡 入力文字列を単一のデータとして扱います。")
            data_list = [input_data]
    elif isinstance(input_data, list):
        print(f"💡 リスト入力されたデータを扱います。")
        data_list = input_data
    # 分かち書き
    return [tagger.parse(text).strip() for text in data_list]

def compute_bleu_from_input(predictions: Union[str, List[str]],
                            references: Union[str, List[str]]) -> pd.DataFrame:
    """
    生成文と正解文を処理してBLEUスコアを計算
    """
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")
    pred_wakati_list = load_data(predictions, tagger)
    ref_wakati_list = load_data(references, tagger)

    if len(pred_wakati_list) != len(ref_wakati_list):
        if len(ref_wakati_list) == 1:
            # 単一参照文なら全ての生成文に対応
            ref_wakati_list = ref_wakati_list * len(pred_wakati_list)
        else:
            raise ValueError(
                f"参照文と生成文の数が一致しません。参照文: {len(ref_wakati_list)}, 生成文: {len(pred_wakati_list)}"
            )

    bleu_scores = [compute_bleu([p], [r]) for p, r in zip(pred_wakati_list, ref_wakati_list)]
    df = pd.DataFrame(bleu_scores)
    return df

# --- 実行例 ---

# 単一文字列
single_ref = "「春」をキーワードに星加アナウンサーが松山・石手を歩く。春の河川敷でビーチバレーの特訓をする若者やこの春入学を控える新一年生、自宅の壁一面にレトロ看板を「貼る」夫婦に遭遇。食欲の春！一日７５０本売れる食パンに、老舗うどん店の「メロンうどん」とは？（写真は一場面）"
single_pred = "春の訪れを感じさせる、石手寺周辺での春祭り企画が行われています。参加者は「私にとっての春」について書いた短い文章を投稿できます。酒造りに関する不思議な伝説や、石手川沿いの造り酒屋栄光酒造の伝統的な日本..."

df_single = compute_bleu_from_input(single_pred, single_ref)
print("✅ スコア結果 (単一文字列):")
display(df_single)
print("-" * 50)

# 複数リストの例
multi_refs = [single_ref, single_ref]
multi_preds = [single_pred, single_pred]
df_multi = compute_bleu_from_input(multi_preds, multi_refs)
print("✅ スコア結果 (リスト):")
display(df_multi)
print("-" * 50)

# ファイルパス例
file_ref = "answer.txt"
file_pred = "mt5.txt"
try:
    df_file = compute_bleu_from_input(file_pred, file_ref)
    print("✅ スコア結果 (ファイル):")
    display(df_file)
except FileNotFoundError:
    print("ファイルが存在しません。'answer.txt'と'gemma12b.txt'を作成して試してください。")

# ファイルパス例（全体をまとめてBLEU）
try:
    # IPAdic + MeCab
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")

    # ファイルを読み込んで全体を1つの文字列にまとめる
    with open(file_ref, 'r', encoding='utf-8') as f:
        ref_text = " ".join([line.strip() for line in f if line.strip()])
    with open(file_pred, 'r', encoding='utf-8') as f:
        pred_text = " ".join([line.strip() for line in f if line.strip()])

    # 分かち書き
    ref_wakati = tagger.parse(ref_text).strip()
    pred_wakati = tagger.parse(pred_text).strip()

    # BLEU計算（全体1つとして）
    from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
    smoothing = SmoothingFunction().method1
    score = corpus_bleu([[ref_wakati.split()]], [pred_wakati.split()], smoothing_function=smoothing)

    print("✅ 全体BLEUスコア:")
    print(round(score, 3))

except FileNotFoundError:
    print("ファイルが存在しません。'answer.txt'と'gemma12b.txt'を作成して試してください。")


💡 入力文字列を単一のデータとして扱います。
💡 入力文字列を単一のデータとして扱います。
✅ スコア結果 (単一文字列):


,BLEU
0,0.01


--------------------------------------------------
💡 リスト入力されたデータを扱います。
💡 リスト入力されたデータを扱います。
✅ スコア結果 (リスト):


,BLEU
0,0.01
1,0.01


--------------------------------------------------
💡 入力文字列を単一のデータとして扱います。
💡 入力文字列を単一のデータとして扱います。
✅ スコア結果 (ファイル):


,BLEU
0,0.17


ファイルが存在しません。'answer.txt'と'gemma12b.txt'を作成して試してください。
